In [1]:
import sys

sys.path.append('..')

from action_language_interpreter.action_language import *

# Instance

In [2]:
nr_battery_levels = 3
nr_positions = 3
goal_position = 2
AD = ActionDescription()

In [3]:
battery_levels = [FluentLiteral(f"battery({battery_level})") for battery_level in range(nr_battery_levels)]
positions = [FluentLiteral(f"position({position})") for position in range(nr_positions)]
objective_reached = FluentLiteral('objective_reached')
objective_reachable = FluentLiteral('objective_reachable')

In [4]:
battery_levels

[battery(0), battery(1), battery(2)]

In [5]:
positions

[position(0), position(1), position(2)]

In [6]:
events: Set[Event] = {'move', 'wait'}

# Semi-Automatic Grounding

In [7]:
for b,battery_level in enumerate(battery_levels):
    for e,event in enumerate(events):
        if b != 0:
            AD.add_stmt_causes_if(event, battery_levels[b-1], battery_level)
            AD.add_stmt_causes_if(event, ~battery_level, battery_level)
    for other_battery_level in battery_levels:
        if battery_level != other_battery_level:
            AD.add_stmt_if(~battery_level, other_battery_level)

In [8]:
for p,position in enumerate(positions):
    if p < len(positions) - 1:
        AD.add_stmt_causes_if('move', positions[p+1], position)
        AD.add_stmt_causes_if('move', ~position, position)
    AD.add_stmt_causes_if('wait', position, position)
    for b, battery_level in enumerate(battery_levels):
        if b + p >= 2:
            AD.add_stmt_if(objective_reachable, position, battery_level)
        else:
            AD.add_stmt_if(~objective_reachable, position, battery_level)
    for other_position in positions:
        if position != other_position:
            AD.add_stmt_if(~position, other_position)

In [9]:
AD.add_stmt_impossible_if('move', positions[2])
AD.add_stmt_impossible_if('move', battery_levels[0])
AD.add_stmt_if(~objective_reached, ~positions[2])
AD.add_stmt_if(objective_reached, positions[2], ~objective_reached)

In [10]:
AD

({wait causes ¬battery(2) if battery(2), move causes ¬battery(1) if battery(1), wait causes position(0) if position(0), wait causes battery(0) if battery(1), wait causes battery(1) if battery(2), move causes position(1) if position(0), move causes position(2) if position(1), wait causes position(1) if position(1), move causes ¬battery(2) if battery(2), wait causes position(2) if position(2), move causes battery(0) if battery(1), move causes battery(1) if battery(2), move causes ¬position(1) if position(1), wait causes ¬battery(1) if battery(1), move causes ¬position(0) if position(0)}, {¬position(2) if position(0), ¬battery(1) if battery(0), ¬objective_reachable if battery(0),position(0), objective_reachable if battery(2),position(2), ¬battery(2) if battery(1), ¬position(0) if position(1), ¬position(1) if position(2), objective_reachable if battery(1),position(1), ¬battery(0) if battery(2), objective_reachable if battery(2),position(0), objective_reached if ¬objective_reached,position(

# Execution

In [11]:
initial_state, relevant_compliance_rules = AD.comply({battery_levels[2], positions[0]})
initial_state

{¬battery(0),
 ¬battery(1),
 battery(2),
 objective_reachable,
 ¬objective_reached,
 position(0),
 ¬position(1),
 ¬position(2)}

In [12]:
AD.executable_events(initial_state, events)

({'move', 'wait'}, ())

## Unsuccessfull Path

In [13]:
u_state1, u_relevant_stmts1 = AD.execute(initial_state, 'wait')

In [14]:
u_state1

{¬battery(0),
 battery(1),
 ¬battery(2),
 ¬objective_reachable,
 ¬objective_reached,
 position(0),
 ¬position(1),
 ¬position(2)}

In [15]:
u_relevant_stmts1

[wait causes ¬battery(2) if battery(2),
 wait causes position(0) if position(0),
 wait causes battery(1) if battery(2),
 ¬position(2) if position(0),
 ¬battery(2) if battery(1),
 ¬battery(0) if battery(1),
 ¬objective_reachable if battery(1),position(0),
 ¬objective_reached if ¬position(2),
 ¬position(1) if position(0)]

In [16]:
AD.executable_events(u_state1, events)

({'move', 'wait'}, ())

In [17]:
u_state2, u_relevant_stmts2 = AD.execute(u_state1, 'move')

In [18]:
u_state2

{battery(0),
 ¬battery(1),
 ¬battery(2),
 ¬objective_reachable,
 ¬objective_reached,
 ¬position(0),
 position(1),
 ¬position(2)}

In [19]:
u_relevant_stmts2

[move causes ¬battery(1) if battery(1),
 move causes position(1) if position(0),
 move causes battery(0) if battery(1),
 move causes ¬position(0) if position(0),
 ¬battery(1) if battery(0),
 ¬position(0) if position(1),
 ¬position(2) if position(1),
 ¬battery(2) if battery(0),
 ¬objective_reached if ¬position(2),
 ¬objective_reachable if battery(0),position(1)]

In [20]:
u_scenario_path = (initial_state, [('wait', u_state1), ('move', u_state2)])

In [21]:
AD.executable_events(u_state2, events)

({'wait'}, (move impossible_if battery(0),))

# Explanations

In [22]:
direct_effects, indirect_effects = get_first_causal_explanation(action_description=AD, scenario_path=u_scenario_path, outcome={~objective_reachable})

In [23]:
direct_effects

{battery(1), ¬battery(2), position(0)}

In [24]:
indirect_effects

{¬objective_reachable}

In [25]:
supporting_literals, initial_literals = get_second_causal_explanation(action_description=AD, scenario_path=u_scenario_path, outcome={~objective_reachable}, effects=direct_effects | indirect_effects)

In [26]:
supporting_literals

set()

In [27]:
initial_literals

set()

In [28]:
supporting_events, uncaused_literals = get_third_causal_explanation(action_description=AD, scenario_path=u_scenario_path, outcome={~objective_reachable})

In [29]:
supporting_events

set()

In [30]:
uncaused_literals

set()